Installing the Julep Client

In [1]:
!pip install julep -U --quiet

In [2]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

## creating julep client with the api key


In [4]:
from julep import Client
import os

api_key = os.getenv("JULEP_API_KEY")

# Create a Julep client
client = Client(api_key=api_key, environment="production")

### Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the Agent section in [Julep Concepts](https://docs.julep.ai/docs/concepts/agents).

In [5]:
# Defining the agent
name = "Browser Use Assistant"
about = "an assistant that can interact with a web browser to perform tasks on behalf of users."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="claude-3.5-sonnet-20241022",
)

### Creating an an "tool"

Tools are the object which define the actions that the agent can perform.

To learn more about the tools, please refer to the Tools section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tools).

In [6]:
# Adding the tools to the agent
try:
    client.agents.tools.create(
        agent_id=AGENT_UUID,
        **{
            "name": "computer",
            "type": "computer_20241022",
            "computer_20241022": {
                "display_height_px": 768,
                "display_width_px": 1024,
                "display_number": 1,
            },
        }
    )
except Exception as e:
    print("Already added")

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [7]:
import yaml
# please replace the DEMO_API_KEY and DEMO_PROJECT_ID with your own API Key and Project ID of BrowserBase

# Defining the task
task_def = yaml.safe_load('''
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: computer-use-task
description: A task that uses a computer to navigate the web.

########################################################
################### INPUT SCHEMA #######################
########################################################
input_schema:
  type: object
  properties:
    goal:
      type: string
  required:
    - goal

########################################################
####################### TOOLS ##########################
########################################################

tools:
- name: create_browserbase_session
  type: integration
  integration:
    provider: browserbase
    method: create_session
    setup:
      api_key: "YOUR_BROWSERBASE_API_KEY"
      project_id: "YOUR_BROWSERBASE_PROJECT_ID"

- name: get_session_view_urls
  type: integration
  integration:
    provider: browserbase
    method: get_live_urls
    setup:
      api_key: "YOUR_BROWSERBASE_API_KEY"
      project_id: "YOUR_BROWSERBASE_PROJECT_ID"

- name: perform_browser_action
  type: integration
  integration:
    provider: remote_browser
    method: perform_action
    setup:
      width: 1024
      height: 768

- name: create_julep_session
  type: system
  system:
    resource: session
    operation: create

- name: session_chat
  type: system
  system:
    resource: session
    operation: chat

########################################################
################### MAIN WORKFLOW ######################
########################################################

main:

# Step #0 - Create Julep Session
- tool: create_julep_session
  arguments:
    agent: $ str(agent.id)
    situation: "Juelp Browser Use Agent"
    recall: 'False'

# Step #1 - Store Julep Session ID
- evaluate:
    julep_session_id: $ _.id

# Step #2 - Create Browserbase Session
- tool: create_browserbase_session
  arguments:
    project_id: "c35ee022-883e-4070-9f3c-89607393214b"

# Step #3 - Store Browserbase Session Info
- evaluate:
    browser_session_id: $ _.id
    connect_url: $ _.connect_url

# Step #4 - Get Session View URLs
- tool: get_session_view_urls
  arguments:
    id: $ _.browser_session_id

# Step #5 - Store Debugger URL
- evaluate:
    debugger_url: $ _.urls.debuggerUrl

# Step #6 - Navigate to Google
# Navigate to google to avoid sending a blank 
# screenshot when computer use starts
- tool: perform_browser_action
  arguments:
    connect_url: $ steps[3].output.connect_url
    action: "navigate"
    text: "https://www.google.com"

# Step #7 - Run Browser Workflow
- workflow: run_browser
  arguments:
    julep_session_id: $ steps[1].output.julep_session_id
    cdp_url: $ steps[3].output.connect_url
    messages:
    - role: "user"
      content: |-
        $ f"""
        <SYSTEM_CAPABILITY>
        * You are utilising a headless chrome browser to interact with the internet.
        * You can use the computer tool to interact with the browser.
        * You have access to only the browser.
        * You are already inside the browser.
        * You can't open new tabs or windows.
        * For now, rely on screenshots as the only way to see the browser.
        * You don't have access to the browser's UI.
        * YOU CANNOT WRITE TO THE SEARCH BAR OF THE BROWSER.
        </SYSTEM_CAPABILITY>
        <GOAL>
        * + {steps[0].input.goal} + NEWLINE + </GOAL> """
    workflow_label: "run_browser" # <----- REMOVE THIS


########################################################
################# RUN BROWSER SUBWORKFLOW ##############
########################################################

run_browser:

# Step #0 - Agent Interaction
- tool: session_chat
  arguments:
    session_id: $ _.julep_session_id
    messages: $ _.messages
    recall: $ False

# Step #1 - Evaluate the response from the agent
- evaluate:
    content: $ _.choices[0].message.content
    tool_calls: |-
      $ [ 
        { 
          'tool_call_id': tool_call.id, 
          'action': load_json(tool_call.function.arguments)['action'], 
          'text': load_json(tool_call.function.arguments).get('text'), 
          'coordinate': load_json(tool_call.function.arguments).get('coordinate') 
        } 
        for tool_call in _.choices[0].message.tool_calls or [] if tool_call.type == 'function']


# Step #2 - Perform the actions requested by the agent
- foreach:
    in: $ _.tool_calls
    do:
      tool: perform_browser_action
      arguments:
        connect_url: $ steps[0].input.cdp_url
        action: $ _.action if not (str(_.get('text', '')).startswith('http') and _.action == 'type') else 'navigate'
        text: $ _.get('text')
        coordinate: $ _.get('coordinate')

# Step #3 - Convert the result of the actions into a chat message
- evaluate:
    contents: >-
      $ [ \
        { \
          'type': 'image_url', \
          'image_url': { \
            'url': result['base64_image'], \
          } \
        } if result['base64_image'] is not None else \
        { \
          'type': 'text', \
          'text': result['output'] if result['output'] is not None else 'done' \
        } \
        for result in _]

# Step #4 - Convert the result of the actions into a chat message
- evaluate:
    messages: "$ [{'content': [_.contents[i]], 'role': 'tool', 'name': 'computer', 'tool_call_id': steps[1].output.tool_calls[i].tool_call_id} for i in range(len(_.contents))]"

# Step #5 - Check if the goal is achieved and recursively run the browser
- workflow: check_goal_status
  arguments:
    messages: $ _.messages
    julep_session_id: $ steps[0].input.julep_session_id
    cdp_url: $ steps[0].input.cdp_url
    workflow_label: "check_goal_status" 


########################################################
############## CHECK GOAL STATUS SUBWORKFLOW ##############
########################################################

check_goal_status:

# Step #0 - Check if the goal is achieved and recursively run the browser
- if: $ len(_.messages) > 0
  then:
    workflow: run_browser
    arguments:
      messages: $ _.messages
      julep_session_id: $ _.julep_session_id
      cdp_url: $ _.cdp_url
      workflow_label: "run_browser" 
''')


In [8]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def,
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [9]:
execution = client.executions.create(
    task_id=task.id,
     input={
        "agent_id": str(AGENT_UUID),
        "goal": "Navigate to JulepAI's Github repository and tell me the number of stars it has. Remember bro, the link for julep's repository is https://github.com/julep-ai/julep",
        }
)

print("Started an execution. Execution ID:", execution.id)

Started an execution. Execution ID: 067a63c3-638a-77e6-8000-3cd61baa9ebb


## Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [15]:
import pprint
import time

def reformat_output(output):
    if isinstance(output, dict):
        for key, value in output.items():
            if (key == "base64_image" or key == "url") and value is not None:
                output[key] = value[:20] + '...'
            else:
                output[key] = reformat_output(value)
    elif isinstance(output, list):
        for i, item in enumerate(output):
            output[i] = reformat_output(item)
    return output
# List all the task steps that have been executed
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for index, transition in enumerate(reversed(transitions)):
    print("Transtion Index: ", index)
    print("Transition Type: ", transition.type)
    output = transition.output
    pprint.pprint(reformat_output(output))
    print("----"*100)

Transtion Index:  0
Transition Type:  init
{'agent_id': '5113a738-f5a0-4b71-83e7-b98656985c92',
 'goal': "Navigate to JulepAI's Github repository and tell me the number of "
         "stars it has. Remember bro, the link for julep's repository is "
         'https://github.com/julep-ai/julep'}
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Transtion Index:  1
Transition Type:  step
{'auto_run_tools': False,
 'context_overflow': None,
 'created_at': '2025-02-07T17:00:38.457752Z',
 'forward_tool_calls': False,
 'id': '067a63c3-6752-7225-8000-754d7a2a6056',
 'kind': None,
 'metadata': {},
 'recall_options': {'alpha': 0.7,
                   

## Related Concepts

- [Agents](/concepts/agents)
- [Tasks](/concepts/tasks)
- [Tools](/concepts/tools)
